In [1]:
import getpreparedata.getprepdata as gpd
import models.model as m
import models as mdls
import trader.trader_ml as t

Using TensorFlow backend.


In [4]:
import configs.EUR_PLN_2 as cfginst
import sys
sys.path.append('../')
import configs.config as cfg
from common import utils as u
import pickle
import logging

In [8]:
from tensorflow import keras
import os
import numpy as np
import pandas as pd


In [7]:
instrument = cfginst.instrument
# get or generate datafiles files and folders, if do not exist
namefiles_dict = {}
namefiles_dict = u.creates_filenames_dict(cfginst.instrument, 
                                          namefiles_dict, cfg)

## Get Historical Data that will be used for Training

In [ ]:
odc = gpd.OandaDataCollector(instrument=cfginst.instrument,
                         labels=cfginst.labels,
                         features=cfginst.features,
                         conf_file=cfg.conf_file,
                         namefiles_dict=namefiles_dict)
print('OandaDataCollector object created for instrument {}'.format(
    cfginst.instrument))
NEW_DATA = True
if NEW_DATA:
    # actual data collection of most recent data
    print('OandaDataCollector data collection starts...')
    odc.get_most_recent(granul=cfginst.granul, days = cfginst.days)
    odc.make_features()
    odc.make_lagged_features(lags=cfginst.lags)
    odc.resample_data(brl = cfginst.brl)
    odc.make_3_datasets()
    odc.standardize()
    odc.save_to_file()

    print("All row data downloaded from Oanda for instrument {}".format(
        cfginst.instrument))
    print(odc.raw_data.info(),  end="\n  ******** \n")
    print("Re-sampled data for bar length {} from Oanda for instrument {}".format(
                                                            cfginst.brl, cfginst.instrument))
    print(odc.raw_data_featured_resampled.info(),  end="\n  ******** \n")
else:
    print('OandaDataCollector data is loading from disk...')
    odc.load_data_from_file()
    odc.report()

## Training on the data obtained

In [ ]:
# Todo: do this selection better and not via string. This should reference via dict to the model
model_id = "dnn1"
model_trainer = mdls.trainer.DL_Trainer(cfginst, model_id)

logging.info("Loading data and creating the NN model...")
model_trainer.load_train_data()
model_trainer.set_model()

logging.info("Training the NN model...")
model_trainer.train_model(epochs=70)

logging.info("Evaluating the NN model...")
model_trainer.evaluate_model()

logging.info("Predictions with the NN model...")
model_trainer.make_predictions()

logging.info("Saving the NN model...")
model_trainer.save_model()


## Trade or Test Strategy

In [9]:
# get or generate datafiles files and folders, if do not exist
namefiles_dict = {}
namefiles_dict = u.creates_filenames_dict(instrument, namefiles_dict, cfg)
#load params for data standardization
params = pickle.load(open(namefiles_dict["params"], "rb"))
mu = params["mu"]
std = params["std"]
# load trained model
model = keras.models.load_model(cfg.trained_models_path +
                                instrument + "/DNN_model.h5")
# create trader object using instrument configuration details

In [10]:
print("Layers of model being used are: ")
print(model.layers)
trader = t.DNNTrader(cfg.conf_file,
                   instrument=instrument,
                   bar_length=cfginst.brl,
                   window=cfginst.window,
                   lags=cfginst.lags,
                   units=cfginst.units,
                   model=model,
                   mu=mu, std=std,
                   hspread_ptc=cfginst.hspread_ptc,
                   sma_int=cfginst.sma_int,
                   features=cfginst.features,
                   h_prob_th=cfginst.higher_go_long,
                   l_prob_th=cfginst.lower_go_short)

# either live trading or testing (back or fw testing)
TRADING = 0
BCKTESTING, FWTESTING = (1,0) if not TRADING else (0,0)

if TRADING:
    trader.get_most_recent(days=cfginst.days_inference, granul=cfginst.granul)  # get historical data
    logging.info("main: most recent historical data obtained and resampled" +
                 "now starting streaming data and trading...")
    trader.stream_data(cfginst.instrument, stop=cfginst.stop_trading)  # streaming & trading here!!!!

    if trader.position != 0:
        print("Closing position as we are ending trading!")
        close_order = trader.create_order(instrument=cfginst.instrument,
                                          units=-trader.position * trader.units,
                                          suppress=True, ret=True)  # close Final Position
        trader.report_trade(close_order, "GOING NEUTRAL")  # report Final Trade
else: # TESTING
    # loading data
    assert os.path.exists(namefiles_dict["base_data_folder_name"]), "Base data folder DO NOT exists!"
    train_data = pd.read_csv(namefiles_dict["train_filename"],
                             index_col="time", parse_dates=True, header=0)
    test_data = pd.read_csv(namefiles_dict["test_filename"],
                            index_col="time", parse_dates=True, header=0)
    # valid not used for now, using keras support but that uses
    # std and mean computed on the train+valid data
    train_labels = pd.read_csv(namefiles_dict["train_labl_filename"],
                               index_col="time", parse_dates=True, header=0)
    test_labels = pd.read_csv(namefiles_dict["test_labl_filename"],
                              index_col="time", parse_dates=True, header=0)

    #trader.prepare_data() ### necessary? maybe not if I take data prepared by getpreparedata.py
    if BCKTESTING:
        trader.test(train_data, train_labels)

    else: # fwtesting
        trader.test(test_data, test_labels)

Layers of model being used are: 
[<tensorflow.python.keras.layers.core.Dense object at 0x10701bc50>, <tensorflow.python.keras.layers.core.Dropout object at 0x1070445d0>, <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x140dea1d0>, <tensorflow.python.keras.layers.core.Dense object at 0x107044450>, <tensorflow.python.keras.layers.core.Dropout object at 0x140deabd0>, <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x1183d3150>, <tensorflow.python.keras.layers.core.Dense object at 0x1430d9d90>, <tensorflow.python.keras.layers.core.Dropout object at 0x140dea490>, <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x1431115d0>, <tensorflow.python.keras.layers.core.Dense object at 0x143171650>]



SEQUENCE: predict


outperf is  0.19703783178209167
plotting cumulative results of buy&hold and strategy


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



INFO:root:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/Users/ANDREA/anaconda/envs/oanda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-e9ccc726bb62>", line 49, in <module>
    trader.test(train_data, train_labels)
  File "/Users/ANDREA/PycharmProjects/OandaTrading/trader/trader_ml.py", line 113, in test
    plt.show()
  File "/Users/ANDREA/anaconda/envs/oanda/lib/python3.7/site-packages/matplotlib/pyplot.py", line 353, in show
    return _backend_mod.show(*args, **kwargs)
  File "/Users/ANDREA/anaconda/envs/oanda/lib/python3.7/site-packages/matplotlib/backend_bases.py", line 3544, in show
    cls.mainloop()
  File "/Users/ANDREA/anaconda/envs/oanda/lib/python3.7/site-packages/matplotlib/backends/_backend_tk.py", line 916, in mainloop
    managers[0].window.mainloop()
  File "/Users/ANDREA/anaconda/envs/oanda/lib/python3.7/tkinter/__init__.py", line 1283, in mainloop
    self.tk.m

TypeError: object of type 'NoneType' has no len()